In [30]:
from fishsense_api_sdk.client import Client
import asyncio
from pathlib import Path
from label_studio_sdk.client import LabelStudio
from urllib.parse import urlparse
from fishsense_api_sdk.models.species_label import SpeciesLabel

In [2]:
URL = "http://localhost:8000"

In [11]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [13]:
fs = Client(URL)
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [45]:
DIVE_TO_LABEL_STUDIO_MAP = {
    471: 63,
    466: 62,
    465: 61,
    347: 60,
    349: 59,
    341: 58,
    279: 57,
}

In [46]:
for dive_id, label_studio_project_id in DIVE_TO_LABEL_STUDIO_MAP.items():
    dive = await fs.dives.get(dive_id=dive_id)
    images = await fs.images.get(dive_id=dive_id)

    species_labels = await asyncio.gather(*[fs.labels.get_species_label(image.id) for image in images])
    species_labels = [label for label in species_labels if label is not None]

    if len(species_labels) != len(images):
        # Missing species labels
        break

    species_labels_without_project_ids = [label for label in species_labels if label.label_studio_project_id is None]
    if len(species_labels_without_project_ids) > 0:
        # Some species labels missing project IDs
        break

    dive = None

dive

Dive(id=279, name='111323_Alligator DeepEast Drift2_FSL05', path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05', dive_datetime=datetime.datetime(2023, 11, 13, 12, 34, 8, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=5, dive_slate_id=None)

In [47]:
images = await fs.images.get(dive_id=dive.id)

len(images), images[0]

(155,
 Image(id=78666, path='drive-download-20240307T1050Z/112023_Alligator/111323_Alligator/111323_Alligator_FSL05/111323_Alligator DeepEast Drift2_FSL05/PB130168.ORF', taken_datetime=datetime.datetime(2023, 11, 13, 12, 3, 31, tzinfo=TzInfo(0)), checksum='7252dfb0db0b855616e45607f58c1c93', is_canonical=True, dive_id=279, camera_id=5))

In [48]:
species_labels = await asyncio.gather(*[fs.labels.get_species_label(image.id) for image in images])
species_labels = [label for label in species_labels if label is not None]

if len(species_labels) != len(images):
    label_studio_project_id = DIVE_TO_LABEL_STUDIO_MAP[dive.id]
    tasks = list(ls.tasks.list(project=label_studio_project_id))

    image_to_species_label_map = {label.image_id:label for label in species_labels}
    image_to_task_map = {urlparse(t.data["image"]).path.split("/")[-1]:t for t in tasks}

    for image in images:
        if image.id not in image_to_species_label_map:
            task = image_to_task_map.get(image.checksum)
            if task is not None:
                species_label = SpeciesLabel(
                    id=None,
                    image_id=image.id,
                    label_studio_task_id=task.id,
                    label_studio_project_id=label_studio_project_id,
                    image_url=task.data["image"],
                    updated_at=None,
                    completed=False,
                    label_studio_json={},
                    user_id=None
                )
                await fs.labels.post_species_label(
                    image.id,
                    species_label
                )

    species_labels = await asyncio.gather(*[fs.labels.get_species_label(image.id) for image in images])
    species_labels = [label for label in species_labels if label is not None]

len(species_labels), species_labels[0]

(155,
 SpeciesLabel(id=1024, label_studio_task_id=222252, label_studio_project_id=57, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/7252dfb0db0b855616e45607f58c1c93', updated_at=None, completed=False, label_studio_json={}, image_id=78666, user_id=None))

In [34]:
species_labels_without_project_ids = [label for label in species_labels if label.label_studio_project_id is None]

len(species_labels_without_project_ids)

0

In [35]:
label_studio_project_id = DIVE_TO_LABEL_STUDIO_MAP[dive.id]

label_studio_project_id

60

In [36]:
for label in species_labels_without_project_ids:
    label.label_studio_project_id = label_studio_project_id

    await fs.labels.post_species_label(
        label.image_id,
        label
    )